# Discrete Speech Resynthesis and Speech Continuation walk-through

Below we will see how to use textless-lib to resynthesis speech and generate speech continuations.

### Prerequisites

We'll need fairseq, textless and a bit of other dependencies.

At the moment there is a caveat that Colab doesn't support numpy versions above 1.21, which are required for some textless-lib functionality.

Here we'll use a small workaround by cloning and using a disk copy of textless. If you're running it locally, you can just uncomment the next line

In [1]:
import sys
sys.version

'3.8.20 | packaged by conda-forge | (default, Sep 30 2024, 17:52:49) \n[GCC 13.3.0]'

In [5]:
# this will ask to restart the runtime as it updated
# an already loaded package -- please restart
! pip install numpy==1.22.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 8.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0


In [3]:
! pip install torch torchaudio AMFM_decompy librosa threadpoolctl==3.0.0 numba==0.53.0 joblib scikit-learn unidecode inflect

  Using cached torch-2.4.1-cp38-cp38-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchaudio-2.4.1-cp38-cp38-manylinux1_x86_64.whl.metadata (6.4 kB)
  Using cached amfm_decompy-1.0.12.2-py3-none-any.whl.metadata (8.6 kB)
  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached threadpoolctl-3.0.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached numba-0.53.0-cp38-cp38-manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached Unidecode-1.4.0-py3-none-any.whl.metadata (13 kB)
  Using cached inflect-7.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached llvmlite-0.36.0-cp38-cp38-manylinux2010_x86_64.whl.metadata (4.9 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.1-py3-none-any.wh

In [6]:
! pip uninstall megaconf hydra-core omegaconf textlesslib -y

In [7]:
! pip uninstall hydra-core -y

In [8]:
! pip install hydra-core==1.0.0

  Using cached hydra_core-1.0.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached antlr4-python3-runtime-4.8.tar.gz (112 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of omegaconf to determine which version is compatible with other requirements. This could take a while.
  Using cached omegaconf-2.2.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached omegaconf-2.2.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached omegaconf-2.2.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached omegaconf-2.1.2-py3-none-any.whl.metadata (3.6 kB)
  Using cached PyYAML-6.0.3-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.1 kB)
Using cached hydra_core-1.0.0-py3-none-any.whl (120 kB)
Using cached omegaconf-2.1.2-py3-none-any.whl (74 kB)
Using cached PyYAML-6.0.3-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (806 kB)
  Created

In [9]:
! pip install pip==24.0


  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1


In [10]:
! pip uninstall omegaconf -y
! pip install omegaconf==2.0.1

Found existing installation: omegaconf 2.1.2
Uninstalling omegaconf-2.1.2:
  Successfully uninstalled omegaconf-2.1.2


  Using cached omegaconf-2.0.1-py3-none-any.whl.metadata (3.0 kB)
Using cached omegaconf-2.0.1-py3-none-any.whl (36 kB)
DEPRECATION: omegaconf 2.0.1 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
! export FAIRSEQ_USE_FAST=0

In [2]:
! pip uninstall fairseq -y
! pip install --no-cache-dir --no-build-isolation git+https://github.com/pytorch/fairseq.git@dd106d9534b22e7db859a6b87ffd7780c38341f8
! git clone https://github.com/facebookresearch/textlesslib.git && cd textlesslib && pip install -e .

Found existing installation: fairseq 1.0.0a0+dd106d9
Uninstalling fairseq-1.0.0a0+dd106d9:
  Successfully uninstalled fairseq-1.0.0a0+dd106d9
  Cloning https://github.com/pytorch/fairseq.git (to revision dd106d9534b22e7db859a6b87ffd7780c38341f8) to /tmp/pip-req-build-6cbkr00a
  Running command git clone --filter=blob:none --quiet https://github.com/pytorch/fairseq.git /tmp/pip-req-build-6cbkr00a
  Running command git rev-parse -q --verify 'sha^dd106d9534b22e7db859a6b87ffd7780c38341f8'
  Running command git fetch -q https://github.com/pytorch/fairseq.git dd106d9534b22e7db859a6b87ffd7780c38341f8
  Running command git checkout -q dd106d9534b22e7db859a6b87ffd7780c38341f8
  Resolved https://github.com/pytorch/fairseq.git to commit dd106d9534b22e7db859a6b87ffd7780c38341f8
  Running command git submodule update --init --recursive -q
  From https://github.com/ngoyal2707/Megatron-LM
   * branch            adb23324c222aad0aad89308e70302d996a5eaeb -> FETCH_HEAD
  Preparing metadata (pyproject.tom

In [4]:
! pip uninstall textless -y
! cd textlesslib && pip install -e .

Found existing installation: textless 0.1.0
Uninstalling textless-0.1.0:
  Successfully uninstalled textless-0.1.0
Obtaining file:///home/deepak/gslm/textlesslib
  Preparing metadata (setup.py) ... done
DEPRECATION: omegaconf 2.0.1 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Running setup.py develop for textless

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import IPython.display as ipd
import sys
sys.path.insert(0, '/home/deepak/gslm/textlesslib')

import torch
import torchaudio
import pathlib

from textless.data.speech_encoder import SpeechEncoder
from textless.data.quantized_datasets import QuantizedLibriSpeech
from textless.vocoders.tacotron2.vocoder import TacotronVocoder

## Resynthesis

Firstly, let us configure what dense model and quantizer we will use:

In [3]:
dense_model_name = "hubert-base-ls960"
quantizer_name = "kmeans"
vocab_size = 200 # one of [50, 100, 200]

We can initialise a SpeechEncoder using its name; this way a corresponding checkpoint will be downloaded automatically

In [4]:
encoder = SpeechEncoder.by_name(
    dense_model_name=dense_model_name,
    quantizer_model_name=quantizer_name,
    vocab_size=vocab_size,
    need_f0=False,
    deduplicate=True,
    f0_normalizer=None,
    f0_quantizer=None,
).cuda()

/home/deepak/miniconda3/envs/deep/lib/python3.8/site-packages/fairseq/checkpoint_utils.py:304: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_locati

We will use a LibriSpeech dataset for our example. We can start with a vanilla version of it, load a single example and listen to it:

In [5]:
! mkdir -p datasets

In [6]:
raw_dataset = torchaudio.datasets.LIBRISPEECH(
    root="./datasets",
    url="dev-clean",
    download=True,
)

In [7]:
audio, input_sample_rate, *_ = raw_dataset[7]
audio

tensor([[-0.0003, -0.0006, -0.0006,  ..., -0.0003, -0.0003, -0.0003]])

In [8]:
ipd.Audio(audio, rate=input_sample_rate)

We can encode this audio example using our SpeechEncoder. The encoded audio is represented as a dictionary with key-value pairs:

In [9]:
encoded_audio = encoder(audio)
encoded_audio.keys()

dict_keys(['units', 'durations', 'dense'])

'units' contains the pseudo-unit stream, while 'durations' encodes per-token durations and 'dense' returns the original HuBERT representation of the audio.

Let's have a look how units look:

In [10]:
encoded_audio['units'][:10]

tensor([ 14, 131, 191,  11,  22,  86,  22, 125,  10, 154], device='cuda:0',
       dtype=torch.int32)

In [11]:
encoded_audio['durations'][:10]

tensor([4, 8, 8, 2, 1, 1, 1, 1, 1, 2], device='cuda:0')

Alternatively, textless-lib provides a simple wrapper around it which will return a "textless" representation of datapoints:

In [12]:
dataset = QuantizedLibriSpeech(
    encoder,
    root="./datasets",
    url="dev-clean",
    download=False,
)

Here datapoints are encoded just in the same way:

In [13]:
datum = dataset[7]
datum['units'][:10]

tensor([ 14, 131, 191,  11,  22,  86,  22, 125,  10, 154], dtype=torch.int32)

Now let us initialise a corresponding Tacotron instance with a matching configuration:

In [14]:
vocoder = TacotronVocoder.by_name(
    dense_model_name,
    quantizer_name,
    vocab_size,
).cuda()

/home/deepak/gslm/textlesslib/textless/vocoders/tacotron2/vocoder.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt_dict = torch.load(tacotron_model_path, map_locat

Everythins is ready for resynthesising the unit stream back into the audio:

In [15]:
resynth_audio = vocoder(datum['units'])

/home/deepak/gslm/textlesslib/textless/vocoders/tacotron2/glow.py:269: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  audio = torch.cuda.FloatTensor(spect.size(0),


In [16]:
ipd.Audio(resynth_audio.cpu().numpy(), rate=vocoder.output_sample_rate)

## Speech Continuation

One additional component we need is a unit-level language model. Here we re-use on from the GSLM paper

In [17]:
from sampler import UnitLanguageModelSampler

...and download a pre-trained checkpoint (more checkpoints [here](https://github.com/pytorch/fairseq/tree/main/examples/textless_nlp/gslm/ulm)).

In [18]:
! mkdir -p LM && \
    wget https://dl.fbaipublicfiles.com/textless_nlp/gslm/hubert/lm_km200/hubert200_lm.tgz -O LM/hubert200_lm.tgz && \
    cd LM/ && \
    tar -xvf hubert200_lm.tgz 

--2025-11-26 11:22:04--  https://dl.fbaipublicfiles.com/textless_nlp/gslm/hubert/lm_km200/hubert200_lm.tgz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.161.229.68, 18.161.229.119, 18.161.229.24, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.161.229.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1450463515 (1.4G) [application/gzip]
Saving to: ‘LM/hubert200_lm.tgz’

LM/hubert200_lm.tgz 100%[===================>]   1.35G  21.1MB/s    in 66s     

2025-11-26 11:23:10 (21.0 MB/s) - ‘LM/hubert200_lm.tgz’ saved [1450463515/1450463515]

hubert200_lm/
hubert200_lm/dict.txt
hubert200_lm/checkpoint_best.pt


We take the first 5 seconds of the same audio as a prompt:

In [19]:
prompt = audio[:, :input_sample_rate * 5]
ipd.Audio(prompt, rate=input_sample_rate)

...and encode it into the unit stream and double-check how the resynthesised version sounds like:

In [20]:
encoded = encoder(prompt)
units = encoded['units']
units

tensor([ 14, 131, 191,  11,  22, 125,  22, 125,  10, 154,  46,  49,  50,  12,
         93,  66,  31, 127, 160,  17, 112,  23,  96,  12, 172,  85,  89,  31,
         46, 190,  33,   9,  87, 157,  41, 136,   1, 111,  19, 141, 120, 152,
        133,  57, 113,  28,   1, 151, 192,  87,  19, 152,  36, 162, 166, 191,
          8,  11, 149, 125,   8, 125,  22, 125,  89, 174,  37,  79, 143, 104,
        136, 115, 172,  13, 156,  44, 187,  79, 104, 109,  38, 119,  51, 182,
         93,  66, 196, 128, 199,  33, 169, 136, 172,  71,  31, 144,  61, 198,
         12,  85,  89,  31, 100, 115, 177, 106, 193,  72, 170,  78, 111,  19,
         15,  41, 115,  54, 177, 106, 193, 148,  35,  69, 127, 170,   1,  95,
         30,  39, 152,  36, 149, 197,  20, 125,  20, 137,  92],
       device='cuda:0', dtype=torch.int32)

In [21]:
resynth_prompt = vocoder(units).cpu().numpy()
ipd.Audio(resynth_prompt, rate=vocoder.output_sample_rate)

Now we load our downloaded checkpoint and define some parameters for the LM sampling:

In [22]:
sampler = UnitLanguageModelSampler.from_pretrained(model_name_or_path="LM/hubert200_lm")

/home/deepak/miniconda3/envs/deep/lib/python3.8/site-packages/fairseq/checkpoint_utils.py:304: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_locati

In [23]:
sampling_kwargs = {
    "temperature": 0.7,
    "sampling": True,
    "beam": 1,
    "prefix_size": -1,
    "max_len_a": 0.0,
    "max_len_b": 400,
}

In [24]:
# It is a fairseq-based language model, so it accepts text strings as an input.
unit_str = " ".join(list(map(str, units.tolist())))
sampled_unit_str = sampler.sample([unit_str], **sampling_kwargs)[0]
continuation = torch.tensor([int(x) for x in sampled_unit_str.split()]).cuda()
continuation

tensor([ 14, 131, 191,  11,  22, 125,  22, 125,  10, 154,  46,  49,  50,  12,
         93,  66,  31, 127, 160,  17, 112,  23,  96,  12, 172,  85,  89,  31,
         46, 190,  33,   9,  87, 157,  41, 136,   1, 111,  19, 141, 120, 152,
        133,  57, 113,  28,   1, 151, 192,  87,  19, 152,  36, 162, 166, 191,
          8,  11, 149, 125,   8, 125,  22, 125,  89, 174,  37,  79, 143, 104,
        136, 115, 172,  13, 156,  44, 187,  79, 104, 109,  38, 119,  51, 182,
         93,  66, 196, 128, 199,  33, 169, 136, 172,  71,  31, 144,  61, 198,
         12,  85,  89,  31, 100, 115, 177, 106, 193,  72, 170,  78, 111,  19,
         15,  41, 115,  54, 177, 106, 193, 148,  35,  69, 127, 170,   1,  95,
         30,  39, 152,  36, 149, 197,  20, 125,  20, 137,  92,  14, 131, 132,
         22,  10,  59,  27, 115, 111, 141, 102,  15,  41,  49, 159,  12, 176,
         30,  89,  44, 144, 198,  28,  37, 143, 121,  59,  27, 151, 111,  19,
         15,  41, 115,  54,  13, 156,  73,  80,  60, 101,  24, 1

Now, given this unit-level continuation we can vocode it into the audio:

In [25]:
resynth_continuation = vocoder(continuation).cpu().numpy()[:10 * vocoder.output_sample_rate]
ipd.Audio(resynth_continuation, rate=vocoder.output_sample_rate)

In [26]:
def load_audio(path: str):
    waveform, input_sample_rate = torchaudio.load(path)
    if waveform.ndim == 2:
        waveform = waveform.mean(dim=0)

    waveform = encoder.maybe_resample(waveform, input_sample_rate)
    return waveform, input_sample_rate

def resynth_audio(audio):
    encoded = encoder(audio)
    units = encoded['units']
    return vocoder(units).cpu().numpy()


def resynth_extended(audio):
    encoded = encoder(audio)
    units = encoded['units']

    unit_str = " ".join(list(map(str, units.tolist())))
    sampled_unit_str = sampler.sample([unit_str], **sampling_kwargs)[0]
    continuation = torch.tensor([int(x) for x in sampled_unit_str.split()]).cuda()

    return vocoder(continuation).cpu().numpy()[:10 * vocoder.output_sample_rate]

In [33]:
og_audio, input_sample_rate = load_audio("./038.wav")
ipd.Audio(og_audio, rate=input_sample_rate)

In [34]:
re_audio = resynth_audio(og_audio)
ipd.Audio(re_audio, rate=vocoder.output_sample_rate)

In [35]:
re_con_audio = resynth_extended(og_audio)
ipd.Audio(re_con_audio, rate=vocoder.output_sample_rate)

In [37]:
def safe_audio_check(re_audio):
    # First, inspect your audio tensor
    print(f"re_audio type: {type(re_audio)}")
    print(f"re_audio shape: {getattr(re_audio, 'shape', 'No shape attribute')}")
    print(f"re_audio dtype: {getattr(re_audio, 'dtype', 'No dtype attribute')}")

    # Ensure it's a proper tensor with correct shape
    if not isinstance(re_audio, torch.Tensor):
        re_audio = torch.tensor(re_audio)

    # Ensure the tensor has the correct shape (channels, samples)
    if re_audio.dim() == 1:
        re_audio = re_audio.unsqueeze(0)  # Add channel dimension
    elif re_audio.dim() > 2:
        re_audio = re_audio.squeeze()  # Remove unnecessary dimensions

    # Ensure it's the correct data type (float32)
    if re_audio.dtype != torch.float32:
        re_audio = re_audio.float()
    
    return re_audio


In [38]:
torchaudio.save("resynthesised.wav", safe_audio_check(re_audio), vocoder.output_sample_rate)
torchaudio.save("extended.wav", safe_audio_check(re_con_audio), vocoder.output_sample_rate)

re_audio type: <class 'numpy.ndarray'>
re_audio shape: (121600,)
re_audio dtype: float32
re_audio type: <class 'numpy.ndarray'>
re_audio shape: (220500,)
re_audio dtype: float32


## live

In [30]:
import gradio as gr
import numpy as np

# Global variable to store the recording for the next cell
global_audio_data = None

def save_audio(audio):
    global global_audio_data
    if audio is not None:
        sr, y = audio
        # Gradio returns (sample_rate, numpy_array)
        # We store it for the next cell to access
        global_audio_data = (y, sr) 
        return (sr, y) # Return for playback in UI
    return None

# Create the widget
demo = gr.Interface(
    fn=save_audio,
    inputs=gr.Audio(sources=["microphone"], type="numpy"),
    outputs="audio",
    live=True
)

demo.launch(share=True)

/home/deepak/miniconda3/envs/deep/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://e0060e59082c11fd51.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
raw_waveform, input_sample_rate = global_audio_data
ipd.Audio(raw_waveform, rate=input_sample_rate)